# Action-Pointer Network

## Notebook Description

In this Notebook, we create the necessary layers for the Action-Pointer Decoder of our Neural Network as well as the loss function and metric that we will use.

## Code

In [ ]:
import numpy as np
import os
import pickle
import tensorflow as tf
from tensorflow import keras
from keras.layers import Dense, LSTM, Concatenate, LeakyReLU, Softmax, Dropout
from keras.layers import Lambda, Flatten, Bidirectional, TimeDistributed, Reshape, MultiHeadAttention, LayerNormalization
from keras.activations import tanh
from keras.models import Model, Sequential
import keras.backend
import random
from copy import copy

### Feed Forward Layer

First, we create a Feed Forward Layer that will be used throughout our Neural Network.<br>
It consists consists of:
- A first, inner Dense layer with dimension <i>inner_dim</i> and activation function Leaky ReLU with <i>alpha</i> = 0.1
- A second, outer Dense layer with dimension <i>outer_dim</i> and no activation function.

In [ ]:
class FeedForward(keras.layers.Layer):
    
    def __init__(self, inner_dim, outer_dim, alpha=0.1, name=None,**kwargs):
        super(FeedForward, self).__init__(name=name,**kwargs)
        #dimension of inner dense layer
        self.inner_dim = inner_dim
        #dimension of outer dense layer
        self.outer_dim = outer_dim
        #alpha activation for Leaky ReLU of inner dense layer
        self.alpha = alpha
        #inner dense layer
        self.dense_in = Dense(self.inner_dim)
        #outer dense layer
        self.dense_out = Dense(self.outer_dim)
        
    def call(self, x):
        #apply inner Dense laayer
        x = self.dense_in(x)
        #apply Leaky ReLU as activation function 
        x = LeakyReLU(self.alpha)(x)
        #apply outer Dense layer
        x = self.dense_out(x)
        return x
   
    def get_config(self):
        config = super().get_config()
        config.update({
            "outer_dim": self.outer_dim,
            "inner_dim": self.inner_dim,
            "alpha": self.alpha
        })
        return config

### Pointer Attention

Now, we create the attention mechanism of our Pointer Network. It is based on the Paper "Pointer Networks" in 2015.<br>
A sequence will be given as an input alongside a vector that has been created from that sequence by an LSTM.<br>
Apply a learnable matrix weight matrix <i>W2</i> to this vector.<br>
Then, for every vector of the sequence, apply a learnable weight matrix <i>W1</i>.<br>
Afterwards, sum the results and apply the function <i>tanh</i>.<br>
Finally, take its dot product with a learnable weight vector <i>v</i>.<br>
The output is then a vector of the attentions over all elements of the input sequence.

In [ ]:
class Pointer(keras.layers.Layer):
    
    def __init__(self, dim, name=None,**kwargs):
        super(Pointer, self).__init__(name=name,**kwargs)
        self.dim = dim
        #apply learnable weight matrix W1
        self.W1 = Dense(dim, use_bias=False)
        #apply learnable weight matrix W2
        self.W2 = Dense(dim, use_bias=False)
        #learnable weight vector "v"
        self.V = Dense(1, use_bias=False)
        
    def call(self, enc_outputs, dec_output):
        #apply W1 to element of input sequence
        w1_e = self.W1(enc_outputs)
        #apply W2 to vector generated from LSTM by entire sequence
        w2_d = Reshape((1,-1))(self.W2(dec_output))
        #sum and apply tanh
        tanh_out = tanh(w1_e + w2_d)
        #take dot-product with vector "v"
        attention = self.V(tanh_out)
        #output is vector of attentions over all elements of input sequence
        out = Flatten()(attention)
        return out
    
    def get_config(self):
        config = super().get_config()
        config.update({
            "dim": self.dim
        })
        return config

### Metric and Loss Function

To compile our model, we will define a custom metric and a custom loss function.<br>
The custom metric shall measure how much a decision of our Neural Network costs.<br>
In specific, it denotes how much additional relative costs arise by taking the action estimated to be optimal by our Neural Network in a state instead of taking the true optimal action. For this, we assume that the optimal policy would be applied from each successor state on, so the chosen action of our Neural Network for the current state would be the only disruption of optimality with regards to the schedule.<br>
The metric is therefore defined as the quotient of the Q-value of the indicated and the optimal action.<br>
We do also substract 1 and multiply the factor 100, so that the final number stands for the relative addition in costs as a percentage.

In [ ]:
#y_true are the real normalized Q-values, y_pred the predicted ones
#since the Q-values got normalized, the action with the highest target value corresponds to the lowest Q-value 
def costs(y_true, y_pred):
    
    #take the indices of the actions estimated to be optimal by our Neural Network
    indices = keras.backend.argmax(y_pred, axis=1)
    length = np.shape(indices)[0]
    #get the true normalized target values associated with these actions
    inv_values = keras.backend.eval(y_true)[np.arange(length),indices]
    #the normalized target values are exactly the quotient of the optimal and the chosen Q-values
    #therefore, their inverse denotes exactly the relative additional costs
    Q_factors = 1/inv_values
    #average over all considered states
    Q_factor = np.mean(Q_factors)
    #give only relative cost increase as percentage
    Q_factor = (Q_factor - 1)*100
    
    return Q_factor

For the loss function we will use the mean-squarred-error.<br>
Note, however, that we have not applied a softmax when creating the data nor the Neural Network. Instead, we will do this within the loss function itself, so we have to safe the data in less formats, since we need the target values and predictions before the softmax for our custom metric above. Consequently, we need to define a custom loss function as well.

In [ ]:
def MSE_with_Softmax(y_pred, y_true):
    #apply softmax
    y_pred = Softmax()(y_pred)
    y_true = Softmax()(y_true)
    #loss is MSE
    loss = keras.losses.MeanSquaredError()(y_pred,y_true)
    #loss will be small, so upscale for better readability
    return (loss * 10e4)